## Problem Explanantion

* We generate $X$ from a 3-dimensional normal distribution with an arbitrary positive definite covariance matrix. Then we generate the data through $Y= \beta_0 + \beta^TX + \epsilon$, where $\epsilon \sim N(0,1)$.
* We applied standard linear regression.
* We applied gradient descent by pytorch, using stochastic gradient descent with batch = $N/10$.

In [1]:
import torch
import numpy as np

## Generate a Model

* np.random.multivariate_normal(mean, cov, N) # how we generate a multivariate normal data
* np.matmul(N-by-p-matrix, p-vector) outputs an N-vector.
* N-by-1-matrix + N-vector -> Don't do this, since this does not match up. In adding up, do either matrix + matrix, or vector + vector.
* hstack(input of nd.arrays (with same row size)): This is why there should be double pair of ()'s.
* If nothing else, keep parameters (mu, beta, etc.) to be a float, not an integer, by adding a decimal point.

In [2]:
np.random.seed(0)
N=1000; p=3
mean = np.array([1.,3,5])
W = np.random.uniform(0,1,9).reshape(3,3)
cov = np.matmul(W, W.transpose())

# generate X
Xdat = np.random.multivariate_normal(mean, cov, N) 
ones=np.array([1.]*N)
Xmat=np.hstack((ones.reshape(N,1), Xdat))

# generate Y
beta = np.array([-5.,10,-5,5])
error = np.random.normal(0,1,N)
Yvec = np.matmul(Xmat, beta) + error

## Simple Linear Regression with function

* The most standard way is numpy & sklearn.
* model = LinearRegression() : We must add () in the end. Otherwise it does not work.
* This function creats a blank object called model, and later we add the regression results inside the object.
* We do not need to add a separate constant column. Just put the X-variables without constant.

* sklearn.linear_model.LinearRegression(input, output) : input should not include constant.

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression() # must add () in the end.
reg.fit(Xdat, Yvec) 
print(reg.intercept_) # beta0
print(reg.coef_) # beta coefficients

-5.338628118441758
[ 9.9270631  -5.0296665   5.09387627]


* np.linalg.lstsq(input, output, rcond=None): input (Xmat) should include the constant term.

In [4]:
res=np.linalg.lstsq(Xmat, Yvec, rcond=None)
res[0]

array([-5.33862812,  9.9270631 , -5.0296665 ,  5.09387627])

* We can also use np.linalg.lstsq(Xmat, Yvec, rcond=None).
* https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html

In [5]:
from sklearn.metrics import mean_squared_error
ypred = reg.predict(Xdat)
mean_squared_error(ypred, Yvec)

0.9823059580775446

## Simple Linear Regression with pytorch (SGD)

* When converting from np.array into tensor, we can only do it once, because the input should stay a np.array.
* We should reshape the output vector as N-by-1 matrix.
* .reshape() also applies for tensor objects.
* batch_size should be an integer. Use int(), and it will remove the digits under the decimal points.

* The default dtype in torch is torch.float32 (neither float32 nor torch.float64). But default in numpy is float64, and this causes a compatibility issue. To elaborate, the Xdat object that we converted from np.array is of torch.float64, but model.parameters() have torch.float32, not being compatible.
* Therefore we either have to convert pytorch to have float64, or convert the numpy objects into float32 before converting into tensors.

* The reason for using SGD is as follows. (https://towardsdatascience.com/can-we-use-stochastic-gradient-descent-sgd-on-a-linear-regression-model-e50327b07d33)
* First we cannot use $\hat{\beta}=(X^TX)^{-1}X^T\mathbf{y}$ in the case where $p$ is large. Then it will be difficult to invert $(X^TX)$.
* For this reason we may choose gradient method, since the gradient $\frac{\partial}{\partial \beta} \|\mathbf{y}-X\beta\|^2 = -2X^T\mathbf{y} + 2X^TX\beta$ does not involve the inverse matrix. However this is still a problem when $N$ is large. Sometimes it is so large that we do not have enough memory to store $X\in \mathbb{R}^{N\times p}$ and $\mathbf{y}\in \mathbb{R}^N$. For this reason, we apply SGD, updating $\beta$ with a small number of batch in each iteration.

### Step 1: manage the dtype

* Choice 1: convert the np.arrays into float32 before making it as tensors. .astype : convert the dtype of numpy arrays.

In [6]:
# Xdat = Xdat.astype('float32')
# Yvec = Yvec.reshape(N,1)
# Yvec = Yvec.astype('float32')

# Xdat = torch.from_numpy(Xdat)
# Yvec = torch.from_numpy(Yvec)

* Choice 2: convert the default of torch to torch.float64. 

In [7]:
torch.set_default_dtype(torch.float64)

### step 2: manage the data into torch.nn-applicable forms.

* torch.from_numpy maintains the .dtype: float64 $\rightarrow$ torch.float64.

In [8]:
Yvec = Yvec.reshape(N,1) # convert the vector as an N-by-1-matrix.
Xdat = torch.from_numpy(Xdat)
Yvec = torch.from_numpy(Yvec)

In [9]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(Xdat, Yvec)
from torch.utils.data import DataLoader
batch_size=int(N/10) # convert it into an integer.
train_dl =  DataLoader(train_ds, batch_size, shuffle=True)

### step 3: identify models and objective functions

* When importing torch.nn, two ways are both available. 1) import torch.nn as nn & 2) from torch import nn

In [10]:
# import torch.nn as nn
from torch import nn
torch.manual_seed(0)
model = nn.Linear(p,1) # p variables and 1 output (response)
list(model.parameters()) # initial weight and bias

[Parameter containing:
 tensor([[ 0.5428,  0.2400, -0.0469]], requires_grad=True),
 Parameter containing:
 tensor([0.4858], requires_grad=True)]

In [11]:
import torch.nn.functional as F
loss_fn = F.mse_loss # We will use MSE as our objective function.
opt = torch.optim.SGD(model.parameters(), lr=1e-2)
num_epochs = 1000
torch.manual_seed(1)

### step 4: iterations

* model() needs a matrix input (or 2-dim array) and also outputs a matrix. 
* It takes a data matrix (N $\times$ p) as the input and outputs a value ($N \times 1$). $\rightarrow$ may be $N\times K$ when there are $K$ output values.

In [12]:
# for xb, yb in train_dl:
#     print(xb.shape)
#     print(yb.shape)
#     break

In [13]:
for epoch in range(num_epochs):
    for xb, yb in train_dl:
        pred=model(xb)
        loss=loss_fn(pred, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    if (epoch+1) % 100 ==0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/1000], Loss: 1.7062
Epoch [200/1000], Loss: 0.9917
Epoch [300/1000], Loss: 0.9203
Epoch [400/1000], Loss: 1.0093
Epoch [500/1000], Loss: 1.1624
Epoch [600/1000], Loss: 1.1146
Epoch [700/1000], Loss: 1.0832
Epoch [800/1000], Loss: 0.9810
Epoch [900/1000], Loss: 1.0665
Epoch [1000/1000], Loss: 1.0383


In [14]:
list(model.parameters()) # estimated parameters

[Parameter containing:
 tensor([[10.2462, -5.1813,  4.9443]], requires_grad=True),
 Parameter containing:
 tensor([-4.4185], requires_grad=True)]